# Decision tree

In [1]:
%%html
<!-- tables should be left-aligned-->
<style>
table {margin-left:0 !important}
</style>

## Problem description

### Input

```json
{'wit1': ['a', 'b', 'c', 'a', 'd', 'e'],
 'wit2': ['a', 'e', 'c', 'd'],
 'wit3': ['a', 'd', 'b']
}
```

### Expected output

siglum | token | token | token | token | token | token | token | token
---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ----
wit1  | #start | a | b | c | a | d | e | #end
wit2  | #start | a | e | c | - | d | - | #end
wit3  | #start | a | - | - | - | d | b | #end

### Topological sort

(Partly ordered)

`a (b e) c a d (e b)`

## Load libraries and configure noteook display parameters

In [2]:
import pandas as pd
import collections # for defaultdict
from bitarray import bitarray
import pprint as pp
from prettytable import PrettyTable # not part of anaconda distribution; install with pip
import pandas_profiling # https://towardsdatascience.com/10-simple-hacks-to-speed-up-your-data-analysis-in-python-ec18c6396e6b
import itertools # permutations()
import math # factorial()
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 150)

## Sample data and housekeeping

In [3]:
# sample data with a bit of repetition
witnessData = {'wit1': ['a', 'b', 'c', 'a', 'd', 'e'],
               'wit2': ['a', 'e', 'c', 'd'],
               'wit3': ['a', 'd', 'b']}

# fake stoplist, to ensure that we can identify stopwords and process them last
stoplist = {'a', 'c'} # set

# bitArrays is used to keep track of which witness tokens have already been processed
bitArrays = {k: bitarray(len(witnessData[k])) for k in witnessData}  # create a bitarray the length of each witness
for ba in bitArrays.values():  # initialize bitarrays to all 0 values
    ba.setall(0)

## Create _common sequence table_ (`csTable`)

Record location of all skipgrams in all witnesses.

In [4]:
# csTable: dictionary, in which
#   key: two-item tuple representing skipgram normalized token values (token[0], token[1])
#   value: list of three-item tuples records all locations where the key occurs: (siglum, offset[0], offset[1])
#     In Real Life:
#       values will include the t values corresponding to the normalized token values
#       use a named tuple or dataclass (https://realpython.com/python-data-classes/)
# In this test sample, we find all skip bigrams; in Real Life we would specify parameters for:
#   size of skipgram (bi, tri-, etc.; here bi-)
#   size of window (maximum distance between first and last members of skipgram; here the full witness length)
#   maximum size of skip between members of skipgram (here constrained only by size of window)
csTable = collections.defaultdict(list)
for key, value in witnessData.items(): # key is siglum, value is list of normalized token readings
    # in Real Life the value would also include a non-normalized t property
    for first in range(len(value)): # all first items in bigram
        for second in range(first + 1, len(value)): # pair with all following items
            csTable[(value[first], value[second])].append((key, first, second))
csTable

defaultdict(list,
            {('a', 'b'): [('wit1', 0, 1), ('wit3', 0, 2)],
             ('a', 'c'): [('wit1', 0, 2), ('wit2', 0, 2)],
             ('a', 'a'): [('wit1', 0, 3)],
             ('a', 'd'): [('wit1', 0, 4),
              ('wit1', 3, 4),
              ('wit2', 0, 3),
              ('wit3', 0, 1)],
             ('a', 'e'): [('wit1', 0, 5), ('wit1', 3, 5), ('wit2', 0, 1)],
             ('b', 'c'): [('wit1', 1, 2)],
             ('b', 'a'): [('wit1', 1, 3)],
             ('b', 'd'): [('wit1', 1, 4)],
             ('b', 'e'): [('wit1', 1, 5)],
             ('c', 'a'): [('wit1', 2, 3)],
             ('c', 'd'): [('wit1', 2, 4), ('wit2', 2, 3)],
             ('c', 'e'): [('wit1', 2, 5)],
             ('d', 'e'): [('wit1', 4, 5)],
             ('e', 'c'): [('wit2', 1, 2)],
             ('e', 'd'): [('wit2', 1, 3)],
             ('d', 'b'): [('wit3', 1, 2)]})

## Construct df and add information needed for ordered traversal

In [5]:
# convert to series before df since list lengths vary
csSeries = pd.Series(csTable)
csSeries # creates a NultiIndex, which we will want to flatten

a  b                                [(wit1, 0, 1), (wit3, 0, 2)]
   c                                [(wit1, 0, 2), (wit2, 0, 2)]
   a                                              [(wit1, 0, 3)]
   d    [(wit1, 0, 4), (wit1, 3, 4), (wit2, 0, 3), (wit3, 0, 1)]
   e                  [(wit1, 0, 5), (wit1, 3, 5), (wit2, 0, 1)]
b  c                                              [(wit1, 1, 2)]
   a                                              [(wit1, 1, 3)]
   d                                              [(wit1, 1, 4)]
   e                                              [(wit1, 1, 5)]
c  a                                              [(wit1, 2, 3)]
   d                                [(wit1, 2, 4), (wit2, 2, 3)]
   e                                              [(wit1, 2, 5)]
d  e                                              [(wit1, 4, 5)]
e  c                                              [(wit2, 1, 2)]
   d                                              [(wit2, 1, 3)]
d  b                     

In [6]:
# convert series to dataframe, flatten MultiIndex, label columns
csDf = pd.DataFrame(csSeries).reset_index()
csDf.columns = ["first", "second", "locations"]
csDf

,first,second,locations
0,a,b,"[(wit1, 0, 1), (wit3, 0, 2)]"
1,a,c,"[(wit1, 0, 2), (wit2, 0, 2)]"
2,a,a,"[(wit1, 0, 3)]"
3,a,d,"[(wit1, 0, 4), (wit1, 3, 4), (wit2, 0, 3), (wit3, 0, 1)]"
4,a,e,"[(wit1, 0, 5), (wit1, 3, 5), (wit2, 0, 1)]"
5,b,c,"[(wit1, 1, 2)]"
6,b,a,"[(wit1, 1, 3)]"
7,b,d,"[(wit1, 1, 4)]"
8,b,e,"[(wit1, 1, 5)]"
9,c,a,"[(wit1, 2, 3)]"


In [7]:
# count witnesses for each skipgram (depth of block) and check for uniqueness of skipgram in all witnesses
#   extract sigla inside set comprehension to remove duplicates, then count
csDf["local_witnesses"] = csDf["locations"].map(lambda x: [location[0] for location in x])
csDf["unique_witnesses"]= csDf["local_witnesses"].map(lambda x: set(x))
csDf["local_witnessCount"] = csDf["local_witnesses"].str.len()
csDf["unique_witnessCount"] = csDf["unique_witnesses"].str.len()
csDf["witness_uniqueness"] = csDf["local_witnessCount"] == csDf["unique_witnessCount"]
csDf

,first,second,locations,local_witnesses,unique_witnesses,local_witnessCount,unique_witnessCount,witness_uniqueness
0,a,b,"[(wit1, 0, 1), (wit3, 0, 2)]","[wit1, wit3]","{wit1, wit3}",2,2,True
1,a,c,"[(wit1, 0, 2), (wit2, 0, 2)]","[wit1, wit2]","{wit1, wit2}",2,2,True
2,a,a,"[(wit1, 0, 3)]",[wit1],{wit1},1,1,True
3,a,d,"[(wit1, 0, 4), (wit1, 3, 4), (wit2, 0, 3), (wit3, 0, 1)]","[wit1, wit1, wit2, wit3]","{wit1, wit2, wit3}",4,3,False
4,a,e,"[(wit1, 0, 5), (wit1, 3, 5), (wit2, 0, 1)]","[wit1, wit1, wit2]","{wit1, wit2}",3,2,False
5,b,c,"[(wit1, 1, 2)]",[wit1],{wit1},1,1,True
6,b,a,"[(wit1, 1, 3)]",[wit1],{wit1},1,1,True
7,b,d,"[(wit1, 1, 4)]",[wit1],{wit1},1,1,True
8,b,e,"[(wit1, 1, 5)]",[wit1],{wit1},1,1,True
9,c,a,"[(wit1, 2, 3)]",[wit1],{wit1},1,1,True


In [8]:
# are both tokens are stopwords? (if so, we’ll process them last)
csDf["stopwords"] = csDf[["first","second"]].T.isin(stoplist).all()
csDf

,first,second,locations,local_witnesses,unique_witnesses,local_witnessCount,unique_witnessCount,witness_uniqueness,stopwords
0,a,b,"[(wit1, 0, 1), (wit3, 0, 2)]","[wit1, wit3]","{wit1, wit3}",2,2,True,False
1,a,c,"[(wit1, 0, 2), (wit2, 0, 2)]","[wit1, wit2]","{wit1, wit2}",2,2,True,True
2,a,a,"[(wit1, 0, 3)]",[wit1],{wit1},1,1,True,True
3,a,d,"[(wit1, 0, 4), (wit1, 3, 4), (wit2, 0, 3), (wit3, 0, 1)]","[wit1, wit1, wit2, wit3]","{wit1, wit2, wit3}",4,3,False,False
4,a,e,"[(wit1, 0, 5), (wit1, 3, 5), (wit2, 0, 1)]","[wit1, wit1, wit2]","{wit1, wit2}",3,2,False,False
5,b,c,"[(wit1, 1, 2)]",[wit1],{wit1},1,1,True,False
6,b,a,"[(wit1, 1, 3)]",[wit1],{wit1},1,1,True,False
7,b,d,"[(wit1, 1, 4)]",[wit1],{wit1},1,1,True,False
8,b,e,"[(wit1, 1, 5)]",[wit1],{wit1},1,1,True,False
9,c,a,"[(wit1, 2, 3)]",[wit1],{wit1},1,1,True,True


In [9]:
# sort and update row numbers, so that we can travese the skipgrams as follows
#   (not currently using stopword list to flter)
#   1. Words that don’t repeat within a witness first
#   2. Within that, deepest block (most witnesses) first
#   3. within that, rarest skipgrams first (less repetition is easier to place correctly)
csDf.sort_values(by=["unique_witnessCount", "witness_uniqueness", "local_witnessCount"], ascending=[False, False, True], inplace=True)
csDf.reset_index(inplace=True, drop=True) # update row numbers
csDf

,first,second,locations,local_witnesses,unique_witnesses,local_witnessCount,unique_witnessCount,witness_uniqueness,stopwords
0,a,d,"[(wit1, 0, 4), (wit1, 3, 4), (wit2, 0, 3), (wit3, 0, 1)]","[wit1, wit1, wit2, wit3]","{wit1, wit2, wit3}",4,3,False,False
1,a,b,"[(wit1, 0, 1), (wit3, 0, 2)]","[wit1, wit3]","{wit1, wit3}",2,2,True,False
2,a,c,"[(wit1, 0, 2), (wit2, 0, 2)]","[wit1, wit2]","{wit1, wit2}",2,2,True,True
3,c,d,"[(wit1, 2, 4), (wit2, 2, 3)]","[wit1, wit2]","{wit1, wit2}",2,2,True,False
4,a,e,"[(wit1, 0, 5), (wit1, 3, 5), (wit2, 0, 1)]","[wit1, wit1, wit2]","{wit1, wit2}",3,2,False,False
5,a,a,"[(wit1, 0, 3)]",[wit1],{wit1},1,1,True,True
6,b,c,"[(wit1, 1, 2)]",[wit1],{wit1},1,1,True,False
7,b,a,"[(wit1, 1, 3)]",[wit1],{wit1},1,1,True,False
8,b,d,"[(wit1, 1, 4)]",[wit1],{wit1},1,1,True,False
9,b,e,"[(wit1, 1, 5)]",[wit1],{wit1},1,1,True,False


## Construct topological order of nodes from df information

### `Node` class

In [10]:
class Node(object):
    def __init__(self, norm):
        self.tokendata = {}  # members are tokens (witness:offset pairs); no tokens for start and end nodes
        self.norm = norm # string value of node;
            # here all values are pre-normalized, so the n value on the node is equal to the (implicit) t value on the witness tokens
            # in Real Life, witness tokens will have t values that may differ from their shared n value that appears on the node
        self.rank = None

    def __repr__(self):
        return self.norm

    def __lt__(self, other):  # make it sortable by norm value
        return self.norm < other.norm

    def add_location(self, siglum, offset):
        self.tokendata[siglum] = offset

# TODO: do we need nodes to be sortable by n value?

### Construct topological order (`toList`)

In [11]:
# Decision-tree node
class dtNode(object):
    ###
    # topologically ordered list, dictionary of bitarrays, dataframe with remaining options
    ###
    def __init__(self, _toList, _bitArrays, _df):
        self.toList = _toList
        self.bitArrays = _bitArrays
        self.df = _df
        self.children = []
    def __repr__(self):
        return str(self.toList)

In [12]:
# Root of decision tree inherits empty toList, bitArrays with 0 values, and complete, sorted df
dtRoot = dtNode([], bitArrays, csDf)
dtRoot

[]

In [13]:
# isolate next skipgram to process
current = dtRoot.df.iloc[0,:]
current

first                                                                         a
second                                                                        d
locations              [(wit1, 0, 4), (wit1, 3, 4), (wit2, 0, 3), (wit3, 0, 1)]
local_witnesses                                        [wit1, wit1, wit2, wit3]
unique_witnesses                                             {wit1, wit2, wit3}
local_witnessCount                                                            4
unique_witnessCount                                                           3
witness_uniqueness                                                        False
stopwords                                                                 False
Name: 0, dtype: object

In [14]:
# Identify all possible placements of skipgram
d = collections.defaultdict(list)
for i in current.locations:
    d[i[0]].append(i)
choices = list(itertools.product(*d.values()))
pp.pprint(choices)

[(('wit1', 0, 4), ('wit2', 0, 3), ('wit3', 0, 1)),
 (('wit1', 3, 4), ('wit2', 0, 3), ('wit3', 0, 1))]


In [15]:
# current["first"] and current["second"]
parent = dtRoot
for choice in choices:
    newChild = dtNode(dtRoot.toList.copy(), dtRoot.bitArrays.copy(), dtRoot.df.copy())
    print("\nNew choice:", choice, newChild.bitArrays)
    parent.children.append(newChild)
    for witnessToken in choice:
        print(witnessToken)
        for position, norm in enumerate([current["first"], current["second"]]): # position (0, 1) is first or second skipgram token
            ###
            # what siglum and offset are we looking for?
            ###
            siglum = witnessToken[0]
            offset = witnessToken[position + 1]
            print(siglum, offset, norm)
            ###
            # do we need to process it, or have we alreaady taken care of it?
            ###
            if newChild.bitArrays[siglum][offset]: # already set, so break for this location
                print(siglum, offset, norm, "has already been processed")
                continue
            else:
                print("not processed yet; do it now")
            newChild.bitArrays[siglum][offset] = 1  # record that we've processed this token


New choice: (('wit1', 0, 4), ('wit2', 0, 3), ('wit3', 0, 1)) {'wit1': bitarray('000000'), 'wit2': bitarray('0000'), 'wit3': bitarray('000')}
('wit1', 0, 4)
wit1 0 a
not processed yet; do it now
wit1 4 d
not processed yet; do it now
('wit2', 0, 3)
wit2 0 a
not processed yet; do it now
wit2 3 d
not processed yet; do it now
('wit3', 0, 1)
wit3 0 a
not processed yet; do it now
wit3 1 d
not processed yet; do it now

New choice: (('wit1', 3, 4), ('wit2', 0, 3), ('wit3', 0, 1)) {'wit1': bitarray('100010'), 'wit2': bitarray('1001'), 'wit3': bitarray('110')}
('wit1', 3, 4)
wit1 3 a
not processed yet; do it now
wit1 4 d
wit1 4 d has already been processed
('wit2', 0, 3)
wit2 0 a
wit2 0 a has already been processed
wit2 3 d
wit2 3 d has already been processed
('wit3', 0, 1)
wit3 0 a
wit3 0 a has already been processed
wit3 1 d
wit3 1 d has already been processed


In [ ]:
# traverse rows in order
toList = []
toList.extend([Node('#start'), Node('#end')]) # initialize with start and end nodes, which have no tokens
for row in csDf.itertuples(index=False): # process df rows in order
    ###
    # each row provides first and second (normalized tokens) and offsets of instances of those tokens in witnesses
    # place by associating token, witness, and offset
    ###
    for position, norm in enumerate([row.first, row.second]): # position (0, 1) is first or second skipgram token
        for location in row.locations:
            ###
            # what siglum and offset are we looking for?
            ###
            siglum = location[0]
            offset = location[position + 1]
            ###
            # do we need to process it, or have we alreaady taken care of it?
            ###
            if bitArrays[siglum][offset]: # already set, so break for this location
                continue
            else:
                pass
            ###
            # since we didn’t break, create a new node and figure out where to place it
            ###
            modifyMe = None  # flag that tells us whether we need to modify an existing node (or create a new one)
            floor = 0 # floor and ceiling frame the locations where the new node can be placed
            ceiling = len(toList) - 1
            ###
            # find floor and ceiling
            ###
            for nodePos in range(len(toList)): # determine floor and ceiling by scanning nodes
                currentDict = toList[nodePos].tokendata # keys are a list of witnesses on token
                if siglum not in currentDict.keys():  # this dictionary isn't relevant; look at the next one
                    pass
                else: # is it a new floor or a new ceiling? (we've already filtered out the == case)
                    if currentDict[siglum] < offset: # move up the floor if the new offset is greater than a node already there
                        floor = nodePos + 1
                    else: # we’ve hit the ceiling if the new offset is less than the old one
                        ceiling = nodePos
                        break
            ###
            # scan from floor to ceiling, looking for matching 'norm' value
            #
            # if there is a dictionary to modify, save it as modifyMe (don't modify it yet)
            # TODO: this gets the leftmost if there is more than one, which is not necessarily optimal
            ###
            for pos in range(floor, ceiling):
                if toList[pos].norm == norm:
                    modifyMe = toList[pos]
                    break
            ###
            # if there is a dictionary to modify, do it; otherwise insert a new dictionary at the ceiling
            # TODO: why at the ceiling?
            ###
            if modifyMe is None: # create and insert new token
                new_token = Node(norm)
                new_token.add_location(siglum, offset)
                toList.insert(ceiling, new_token)
            else: # modify existing token
                modifyMe.tokendata[siglum] = offset

            bitArrays[siglum][offset] = 1  # record that we've processed this token

### Build list of edges for each witness

In [ ]:
edgeSets = collections.defaultdict(list)  # key = siglum, value = list of node (source, target) tuples
edgeSourceByWitness = {}  # last target will be next source
for node in toList:  # token.norm is str; token.tokendata is dict with siglum:offset items
    if node.norm == '#start':  # not an edge target, so don’t add an edge, but set up source for next edge
        for siglum in witnessData:
            edgeSourceByWitness[siglum] = node
    elif node.norm == '#end':  # create edges to #end for all witnesses
        for siglum in witnessData:
            edgeSets[siglum].append((edgeSourceByWitness[siglum], node))
    else:
        for key, value in node.tokendata.items():
            # add next witness-specific edge, update value in edgeSourceByWitness
            edgeSets[key].append((edgeSourceByWitness[key], node))
            edgeSourceByWitness[key] = node
edges = set(inner for outer in edgeSets.values() for inner in outer)  # tuples of Tokens

### Index from edge target to source for calculating rank

In [ ]:
findMySources = collections.defaultdict(list)
for edge in edges:
    findMySources[edge[1]].append(edge[0])

### Rank nodes in toList

In [ ]:
for item in toList:
    inEdges = findMySources[item]
    item.rank = max([r.rank for r in inEdges], default=-1) + 1
node_table = pd.DataFrame([(item, item.tokendata, item.rank) for item in toList])
node_table.columns = ["norm", "tokendata", "rank"]
node_table

### Index from rank to nodes for retrieval when building table by columns/ranks

In [ ]:
nodesByRank = collections.defaultdict(list)
for node in toList:
    nodesByRank[node.rank].append(node)

## Create alignment table

In [ ]:
table = PrettyTable(header=False)
orderedSigla = sorted(witnessData.keys())
table.add_column(None,[key for key in orderedSigla])
for rank, nodes in nodesByRank.items():  # add a column for each rank
    columnTokens = {}
    for node in nodes:  # copy tokens from all nodes at rank into a single dictionary; value is string (not offset)
        for key in node.tokendata.keys():
            columnTokens[key] = node.norm                
    columnData = []
    for siglum in orderedSigla:
        if siglum in columnTokens:
            columnData.append(columnTokens[siglum])
        elif node.norm in ["#start", "#end"]:
            columnData.append(node.norm)
        else:
            columnData.append('')
    table.add_column(None, columnData)
print(table)

### Expected output

siglum | token | token | token | token | token | token | token | token
---- | ---- | ---- | ---- | ---- | ---- | ---- | ---- | ----
wit1  | #start | a | b | c | a | d | e | #end
wit2  | #start | a | e | c | - | d | - | #end
wit3  | #start | a | - | - | - | d | b | #end